#COS 485 Character level RNN
The source is based on https://github.com/spro/char-rnn.pytorch

In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag

platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision

In [0]:
!pip install -q tqdm
from tqdm import tqdm

In [0]:
import torch
import math
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from torch.autograd import Variable
import torchvision.transforms as transforms
from IPython import display
import time


In [0]:
use_cuda = False

# Dataset
Download Shakespeare, preprocess and Display some examples

In [6]:
import requests
import string
import random

all_characters = string.printable
n_characters = len(all_characters)

def DownloadFile(url):
    local_filename = url.split('/')[-1]
    r = requests.get(url)
    return r.text

def char_tensor(string):
    tensor = torch.zeros(len(string)).long()
    for c in range(len(string)):
        try:
            tensor[c] = all_characters.index(string[c])
        except:
            continue
    return tensor  

def random_training_set(chunk_len, batch_size, file):
    inp = torch.LongTensor(batch_size, chunk_len)
    target = torch.LongTensor(batch_size, chunk_len)
    for bi in range(batch_size):
        start_index = random.randint(0, len(file) - chunk_len)
        end_index = start_index + chunk_len + 1
        chunk = file[start_index:end_index]
        inp[bi] = char_tensor(chunk[:-1])
        target[bi] = char_tensor(chunk[1:])
    inp = Variable(inp)
    target = Variable(target)
    if use_cuda:
        inp = inp.cuda()
        target = target.cuda()
    return inp, target

def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)
  
target_url = "https://raw.githubusercontent.com/cos495/code/master/shakespeare.txt"
data = DownloadFile(target_url)
#print(random_training_set(10, 8, data))
print(data[10:100])

zen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


#Model
In this code we use Pytorch already implemented Recurrent Neural Network Cell computation with `nn.RNN` and `nn.LSTM`

In [0]:
# https://github.com/spro/char-rnn.pytorch
class CharRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, model="rnn", n_layers=1):
        super(CharRNN, self).__init__()
        self.model = model.lower()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers

        self.encoder = nn.Embedding(input_size, hidden_size)
        self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
        if model=="lstm":
          self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
          
        self.decoder = nn.Linear(hidden_size, output_size)

    def forward(self, input, hidden):
        batch_size = input.size(0)
        encoded = self.encoder(input)
        output, hidden = self.rnn(encoded.view(1, batch_size, -1), hidden)
        output = self.decoder(output.view(batch_size, -1))
        return output, hidden

    def init_hidden(self, batch_size):
        if self.model == "lstm":
            return (Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)),
                    Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size)))
        return Variable(torch.zeros(self.n_layers, batch_size, self.hidden_size))

#Train

###Iinitialize the model

In [0]:
hidden_size = 100
learning_rate = 0.01
cell = "rnn"
n_layers = 2

decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=cell,
    n_layers=n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if use_cuda:
    decoder.cuda()

In [0]:
n_epochs = 2000
chunk_len = 200
print_every = 100
batch_size = 100

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    if use_cuda:
        hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

# Generate Text

In [0]:
def generate(decoder, prime_str='A', predict_len=100, temperature=0.8, cuda=False):
    hidden = decoder.init_hidden(1)
    prime_input = Variable(char_tensor(prime_str).unsqueeze(0))

    if cuda:
        hidden = hidden.cuda()
        prime_input = prime_input.cuda()
    predicted = prime_str

    # Use priming string to "build up" hidden state
    for p in range(len(prime_str) - 1):
        _, hidden = decoder(prime_input[:,p], hidden)
        
    inp = prime_input[:,-1]
    
    for p in range(predict_len):
        output, hidden = decoder(inp, hidden)
        
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = torch.multinomial(output_dist, 1)[0]

        # Add predicted character to string and use as next input
        predicted_char = all_characters[top_i]
        predicted += predicted_char
        inp = Variable(char_tensor(predicted_char).unsqueeze(0))
        if cuda:
            inp = inp.cuda()

    return predicted

In [12]:
start = time.time()
all_losses = []
loss_avg = 0

print("Training for %d epochs..." % n_epochs)
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(chunk_len, batch_size, data))
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print('loss: ', loss)
        print(generate(decoder, 'Wh', 100, cuda=use_cuda), '\n')

  0%|          | 0/2000 [00:00<?, ?it/s]

Training for 2000 epochs...


  5%|▌         | 100/2000 [01:08<21:44,  1.46it/s]

 8s (100 5%) 1.7881]
loss:  1.7881326293945312
Which seine to hear not for the sink now, struch from to whick capparoughy the hering ipon you
I rest  



 10%|█         | 200/2000 [02:16<20:30,  1.46it/s]

...

 18s (1800 90%) 1.4564]
loss:  1.4564405822753905
While here let that house thy master'd how this love stricks, thunkel lose the poor proms,
The people  



 95%|█████████▌| 1900/2000 [21:25<01:07,  1.48it/s]

 25s (1900 95%) 1.4482]
loss:  1.4481936645507814
Whither yet.

KING RICHARD III:
She in proved,
Which that no chilite of sorrow and riege me Murge shal 



100%|██████████| 2000/2000 [22:32<00:00,  1.48it/s]

 32s (2000 100%) 1.4456]
loss:  1.4456103515625
What love and grave?

DUKE VINCENTIO:
That here 'go to the boys you do;
Sab'st both on a presers: poor 




### Let's try sampling with high temperature:

In [13]:
generate(decoder, prime_str="A", temperature= 100, cuda=use_cuda)

"A9\x0c,\rh!a3nBO,cJ1U^#SA|rxu7Ho.-g2|(E.]B*y|8-'3lO-,{=CjXyd'/+@%CVzYuXC)nBrlt~;E`>+\r=:me)+k>u[iHg-:Y5w;j"

### Let's try sampling with low temperature:

In [14]:
generate(decoder, prime_str="A", temperature= 0.001, cuda=use_cuda)

'Abaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaabaab'

### Describe the difference
How do the samples qualitatively change? What does changing the temperature do to distribution of possible outputs?¶

**High temperatures cause the model to take more chances and increase diversity of results, but at a cost of more mistakes. Low temperature will cause the model to make more likely, but also more conservative predictions.** 


### Let's try sampling with reasonable temperature:

In [29]:
generate(decoder, prime_str="A", temperature= 0.25, cuda=use_cuda)

'And shall be his father, and the state to the world, the proceed to the bring the lands the prove the'

**When Temperature is around 0.25, the model generate a reasonable sentence based on the prime_str "A"**

### Example
Insert most meaningful sentence that the network generated, change `prime_str`

In [15]:
generate(decoder, prime_str="why", cuda=use_cuda)

'why, and to my life, sir, we thou hast more good,\nOr shows for a good and king, and high to kill for hi'

### Repeat the sample experiments with nn.LSTM

In [0]:
hidden_size = 100
learning_rate = 0.01
cell = "lstm"
n_layers = 2

decoder = CharRNN(
    n_characters,
    hidden_size,
    n_characters,
    model=cell,
    n_layers=n_layers,
)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

if use_cuda:
    decoder.cuda()

In [0]:
n_epochs = 2000
chunk_len = 200
print_every = 100
batch_size = 100

In [0]:
def train(inp, target):
    hidden = decoder.init_hidden(batch_size)
    if use_cuda:
        hidden = hidden.cuda()
    decoder.zero_grad()
    loss = 0

    for c in range(chunk_len):
        output, hidden = decoder(inp[:,c], hidden)
        loss += criterion(output.view(batch_size, -1), target[:,c])

    loss.backward()
    decoder_optimizer.step()

    return loss.data[0] / chunk_len

In [35]:
start = time.time()
all_losses = []
loss_avg = 0

print("Training for %d epochs..." % n_epochs)
for epoch in tqdm(range(1, n_epochs + 1)):
    loss = train(*random_training_set(chunk_len, batch_size, data))
    loss_avg += loss

    if epoch % print_every == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / n_epochs * 100, loss))
        print('loss: ', loss)
        print(generate(decoder, 'Wh', 100, cuda=use_cuda), '\n')

  0%|          | 0/2000 [00:00<?, ?it/s]

Training for 2000 epochs...


  5%|▌         | 100/2000 [02:29<47:12,  1.49s/it]

 28s (100 5%) 1.3378]
loss:  1.337754669189453
Which we hear me for worthing shall bears me to
epoice of his love, sir: a shin'd admitted of other ch 



 10%|█         | 200/2000 [04:55<44:18,  1.48s/it]

...

 28s (1800 90%) 1.3074]
loss:  1.3074148559570313
Which who be gone with the time and
lices of good contracted as so both be breath.

Second Murderer:
I 



 95%|█████████▌| 1900/2000 [45:52<02:24,  1.45s/it]

 52s (1900 95%) 1.2940]
loss:  1.2940036010742189
What consent anger is thy house;
Shall I would not shall the prince to enjoy him,
Even no more short!  



100%|██████████| 2000/2000 [48:17<00:00,  1.45s/it]

 17s (2000 100%) 1.2852]
loss:  1.28523681640625
Where is yours; but the tincest!

LARTIUS:
How hate my son:
Thou hast truth to the chance that would I 




### Let's try sampling with high temperature:

In [36]:
generate(decoder, prime_str="A", temperature= 100, cuda=use_cuda)

'AmJ%xuz."7!4dZOcMlG#v+//63Rjre-LT/^"zFMN4{@!3#YDi<\nM!!h\r?Cy~0X$lC7g>jiZ4(1(zCHwu\x0cm*Z+Z/(f_^`>)OU**;m='


### Let's try sampling with low temperature:

In [37]:
generate(decoder, prime_str="A", temperature= 0.001, cuda=use_cuda)

'Aaabaaaaabaaaaabaaaaabaaaaabaaaaabaaaaabaaaaacaaaacaaaabaaaaabaaaaabaaaaabaaaaabaaaaabaaaaacaaaacaaaa'


### Let's try sampling with reasonable temperature:

In [41]:
generate(decoder, prime_str="A", temperature= 0.25, cuda=use_cuda)

'And then I have been a cause to the state\nThe sea and the best of the state of his poor brother\nAnd t'

### Cross Entropy per letter in bits and Perplexity results on prediction

#Cell Types

###Elman Cell Computation
An Elman RNN cell with tanh or ReLU non-linearity.

$h' = \tanh(w_{ih} x + b_{ih}  +  w_{hh} h + b_{hh})$

In [0]:
class RNNCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(RNNCell, self).__init__()
        #Implement initializations
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.W_ih = Variable(nn.random_normal([self.input_size, self.hidden_size],
                                       mean=0, stddev=0.2))
        self.W_hh = Variable(nn.random_normal([self.hidden_size, self.hidden_size],
                                       mean=0, stddev=0.2))
        self.b_ih = Variable(nn.zeros([1, self.hidden_size]))
        self.b_hh = Variable(nn.zeros([1, self.hidden_size]))
        self.V = Variable(nn.random_normal([self.hidden_size, self.output_size],
                                       mean=0, stddev=0.2))
        self.b_hy = Variable(nn.zeros([1, self.output_size]))
        
        
    def forward(self, input, hidden):
      
        hidden = nn.sigmoid(torch.matmul(input, self.W_ih) + torch.matmul(hidden, self.W_hh) + self.b_ih + self.b_hh)
        output = torch.matmul(hidden, self.V) + self.b_hy
        
        #Implement forward pass
        return output, hidden

### LSTM Cell Computation
Implement LSTM cell computation described by the following expression

$i_t = \sigma(W_{ii} x_t + b_{ii} + W_{hi} h_{(t-1)} + b_{hi}) \\
f_t = \sigma(W_{if} x_t + b_{if} + W_{hf} h_{(t-1)} + b_{hf}) \\
g_t = \tanh(W_{ig} x_t + b_{ig} + W_{hg} h_{(t-1)} + b_{hg}) \\
o_t = \sigma(W_{io} x_t + b_{io} + W_{ho} h_{(t-1)} + b_{ho}) \\
c_t = f_t c_{(t-1)} + i_t g_t \\
h_t = o_t \tanh(c_t)$


In [0]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1):
        super(LSTMCell, self).__init__()
        #Implement initializations
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        
        self.W_ii = Variable(torch.random_normal([self.hidden_size, self.hidden_size], mean=0, stddev=0.2))
        self.W_hi = Variable(torch.random_normal([self.input_size, self.hidden_size], mean=0, stddev=0.2))
        self.b_ii = Variable(torch.zeros([1, self.hidden_size]))
        self.b_hi = Variable(torch.zeros([1, self.hidden_size]))
        
        self.W_hf = Variable(torch.random_normal([self.hidden_size, self.hidden_size], mean=0, stddev=0.2))
        self.W_if = Variable(torch.random_normal([self.input_size, self.hidden_size], mean=0, stddev=0.2))
        self.b_if = Variable(torch.zeros([1, self.hidden_size]))
        self.b_hf = Variable(torch.zeros([1, self.hidden_size]))
        
        self.W_hg = Variable(torch.random_normal([self.hidden_size, self.hidden_size], mean=0, stddev=0.2))
        self.W_ig = Variable(torch.random_normal([self.input_size, self.hidden_size], mean=0, stddev=0.2))
        self.b_hg = Variable(torch.zeros([1, self.hidden_size]))
        self.b_ig = Variable(torch.zeros([1, self.hidden_size]))
        
        self.W_ho = Variable(torch.random_normal([self.hidden_size, self.hidden_size], mean=0, stddev=0.2))
        self.W_io = Variable(torch.random_normal([self.input_size, self.hidden_size], mean=0, stddev=0.2))
        self.b_ho = Variable(torch.zeros([1, self.hidden_size]))
        self.b_io = Variable(torch.zeros([1, self.hidden_size]))
        
        
    def forward(self, input, hidden, cell):
        #Implement forward pass
        i = nn.sigmoid(torch.matmul(input, self.W_ii) + tf.matmul(hidden, self.W_hi) + self.b_ii + self.b_hi)
        f = nn.sigmoid(torch.matmul(input, self.W_if) + tf.matmul(hidden, self.W_hf) + self.b_if + self.b_hf)
        g = nn.tanh(torch.matmul(input, self.W_ig) + tf.matmul(hidden, self.W_hg) + self.b_ig + self.b_hg)
        o = nn.sigmoid(torch.matmul(input, self.W_io) + tf.matmul(hidden, self.W_ho) + self.b_io + self.b_ho)
        c = torch.mul(f, cell) + torch.mul(i, g)
        h = torch.mul(o, c)
        
        output = o 
        hidden = h
        cell = c
        
        return output, hidden, cell


### LSTM Gates
List all gates that LSTM uses and describe their role

ft : the forget gate layer, It looks the last previous hidden ouptut h(t-1) and the input x, and outputs a number between 0 and 1. A 1
represents “completely keep this” while a 0 represents “completely get rid of this.”

it and gt : the input gate layer, decides which values the network will update. The gate "ft" keep or forget whether the previous info will be kept or not. The gate "it" and "gt" will add to the next output cell. 

ot : the output gate layer, which controls the extent to which the value in the cell is used to compute the output activation of the LSTM unit. 




### Explain how LSTM Cell is different than Simple RNN? (why is it better or worse?)

The difference in structure: Simple RNN has a single neroun like strucutre. But LSTM cell has much more complex structures in a repeating unit, including input gate, forget gate, output gate and memory cell. 

The difference in performance: It is difficult for a standard simple RNN to solve problems that require learning long-term temporal dependencies. LSTM cells include a 'memory cell' that can maintain information in memory for long periods of time. This architecture lets LSTM learn longer-term dependencies. Simple RNNs suffer from vanishing and exploding gradient problems. LSTMs deal with these problems by introducing new gates, such as input and forget gates, which allow for a better control over the gradient flow and enable better preservation of “long-range dependencies”.


#Train CharRNN with your LSTM cell(Extra)